In [3]:
import os
import sys
import pandas as pd
import statsmodels.stats.multitest
import sys

In [ ]:
# read in the result files, skip those tissues with no peer factor generated
tisList = os.listdir("/projects/ps-gymreklab/ydong/data/allTis/")
peerTis = []
count = 0
for currTis in tisList:
    
    for CHR in range(1,23):
        resPath = "/projects/ps-gymreklab/ydong/data/allTis/" +currTis+"/result/chr" + str(CHR) + "Res.csv"
        
        if not os.path.exists(resPath):
            count +=1
#             print(currTis + ": "+str(CHR))
            peerTis.append(currTis)
peerTis = list(set(peerTis))  

In [ ]:
currTis = sys.argv[1]

In [ ]:
if currTis not in peerTis:
    fdr_df = pd.DataFrame(columns = ["chrom", "gene","str.id", "str.start", "n.miss", "slope", "slope_p", "error","error_p",  "pVal","pVal_p","start.dist","stop.dist"])
    chr_df = pd.DataFrame(columns = ["chrom", "gene","str.id", "str.start", "n.miss", "slope", "slope_p", "error","error_p",  "pVal","pVal_p","start.dist","stop.dist"])
    for CHR in range(1,23):
        resPath = "/projects/ps-gymreklab/ydong/data/allTis/" +currTis+"/result/chr" + str(CHR) + "Res.csv"
        chr_df = chr_df.append(pd.read_csv(resPath,index_col= 0).dropna())
        
    chrCol = list(chr_df.columns)
    chrCol.append("newP")


    # bonferroni correction for each exon, following by fdr correction on all exon-STR pairs
    geneL = list(chr_df["gene"].unique())
    spliceDF = pd.DataFrame(columns = chrCol)
    for gene in geneL:
        currSTR = chr_df[chr_df["gene"] == gene]
        rej, corrP,_,_ = statsmodels.stats.multitest.multipletests(list(currSTR["pVal"]), method = "bonferroni")
        currSTR["newP"] = list(corrP)
        bestIdx =currSTR["newP"].idxmin()
        spliceDF = spliceDF.append(currSTR.loc[bestIdx,:])
    rej, correctedP = statsmodels.stats.multitest.fdrcorrection(spliceDF["newP"])
    spliceDF["correctedP"] = correctedP
    fdr_df = fdr_df.append(spliceDF[spliceDF["correctedP"]<=0.05])
        
    fdr_df.to_csv("/projects/ps-gymreklab/ydong/data/allTis/"+currTis + "/fdrRes.csv")
